In [1]:
import os
# os.environ['PATH'] = f"{os.environ['PATH']}:/home/student/.local/bin"
# os.environ['PATH'] = f"{os.environ['PATH']}:/opt/conda/lib/python3.10/site-packages"

os.environ['PATH'] = f"{os.environ['PATH']}:/home/vidy/.local/bin"
os.environ['PATH'] = f"{os.environ['PATH']}:/home/vidy/mambaforge/envs/py310/lib/python3.10/site-packages"


os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

In [ ]:
!python -m pip freeze | grep numpy
!pip install . > /dev/null 2>&1

In [3]:
#Load unity Banana environment
from unityagents import UnityEnvironment
import numpy as np

# Path to the Unity environment binary
env_path = "Banana_Linux/Banana.x86_64"

# Initialize the UnityEnvironment
env = UnityEnvironment(file_name=env_path, no_graphics=False)

brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=False)[brain_name]


# number of actions
action_size = brain.vector_action_space_size

# examine the state space 
state = env_info.vector_observations[0]
state_size = len(state)



Found path: /home/vidy/RL_banana/Value-based-methods/p1_navigation/Banana_Linux/Banana.x86_64
Mono path[0] = '/home/vidy/RL_banana/Value-based-methods/p1_navigation/Banana_Linux/Banana_Data/Managed'
Mono config path = '/home/vidy/RL_banana/Value-based-methods/p1_navigation/Banana_Linux/Banana_Data/MonoBleedingEdge/etc'
Preloaded 'ScreenSelector.so'
Preloaded 'libgrpc_csharp_ext.x64.so'
Unable to preload the following plugins:
	ScreenSelector.so
	libgrpc_csharp_ext.x86.so
Logging to /home/vidy/.config/unity3d/Unity Technologies/Unity Environment/Player.log


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [4]:
import torch
import numpy as np
from dqn_dueling import DQNDueling as DQN
import os

# Hyper parameter
device = 'cuda' 
RUNS_DIR = "runs"
env_id="Banana_Linux"
os.makedirs(RUNS_DIR, exist_ok=True)


fc1_nodes = 512  
MODEL_FILE = os.path.join(RUNS_DIR, f'{env_id}.pt')

# Load the pretrained model
checkpoint = torch.load(MODEL_FILE, map_location=device)  # Ensure compatibility with CPU/GPU
print(f"Loaded pretrained model from {MODEL_FILE}.")
policy_dqn = DQN(state_size, action_size, fc1_nodes).to(device)
policy_dqn.load_state_dict(checkpoint['policy_model_state_dict'])
policy_dqn.eval()  # Set model to evaluation mode

# Evaluation function
def evaluate(env, policy_dqn, n_episodes=5, max_t=1000):
    """Evaluate the agent in the environment."""
    total_score = 0  # Track cumulative score across episodes
    for i_episode in range(1, n_episodes + 1):
        env_info = env.reset(train_mode=False)[brain_name]  # Reset the environment
        state = env_info.vector_observations[0]  # Get the initial state
        score = 0
        for t in range(max_t):
            state_tensor = torch.from_numpy(state).float().unsqueeze(0).to(device)  # Convert state to tensor
            action_values = policy_dqn(state_tensor)  # Get action values from the policy model
            action = torch.argmax(action_values).item()  # Select the action with the highest value

            env_info = env.step(action)[brain_name]  # Take action
            next_state = env_info.vector_observations[0]  # Get next state
            reward = env_info.rewards[0]  # Get reward
            done = env_info.local_done[0]  # Check if episode is done
            score += reward
            state = next_state
            if done:
                break
        total_score += score
        print(f"Episode {i_episode}\tScore: {score}")

    print(f"Average score over {n_episodes} episodes: {total_score / n_episodes}")

# Evaluate the model
evaluate(env, policy_dqn, n_episodes=5, max_t=1000)

# Close the environment
env.close()

Loaded pretrained model from runs/Banana_Linux.pt.
Episode 1	Score: 14.0
Episode 2	Score: 20.0
Episode 3	Score: 17.0
Episode 4	Score: 17.0
Episode 5	Score: 17.0
Average score over 5 episodes: 17.0
